# VacationPy

In [8]:
# Dependencies
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from config import gkey

# Configure gmaps
gmaps.configure(api_key= gkey)

 ## Store Part I results into DataFrame
 - Load the csv exported in Part I to a DataFrame

In [9]:
weather_df = pd.read_csv('../Resources/weather_df.csv')
weather_df

,Unnamed: 0,City,Cloudiness,Country,Humidity,Latitude,Longitude,Max Temperature,Wind Speed
0,0,Hermanus,90,ZA,82,-34.42,19.23,62.01,4.00
1,1,Dubti,0,ET,57,11.74,41.09,76.21,6.51
2,2,Amuntai,1,ID,88,-2.42,115.25,77.36,0.83
3,3,Yangjiang,84,CN,50,21.85,111.97,69.19,7.94
4,4,Atuona,85,PF,76,-9.80,-139.03,77.61,15.79
...,...,...,...,...,...,...,...,...,...
549,549,Necochea,89,AR,82,-38.55,-58.74,66.00,11.99
550,550,Plaster Rock,90,CA,60,46.91,-67.39,39.20,3.36
551,551,Sur,0,OM,51,22.57,59.53,73.27,6.80
552,552,Bukachacha,100,RU,93,52.98,116.92,23.13,1.97


## Humidity Heatmap
- Configure gmaps.
- Use the Lat and Lng as locations and Humidity as the weight.
- Add Heatmap layer to map.
- Create a heat map that displays the humidity for every city from the part I of the homework.

In [10]:
# Grab locations and humidity from weather_df
coord = weather_df[["Latitude", "Longitude"]]
hum = weather_df["Humidity"]


fig = gmaps.figure(zoom_level= 3, center= (35, -20))
heat_layer = gmaps.heatmap_layer(coord, weights= hum, dissipating= False, max_intensity= 100,
                                 point_radius= 1)


fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

## Create new DataFrame fitting weather criteria

Narrow down the DataFrame to find your ideal weather condition:
- A max temperature lower than 80 degrees but higher than 70.
- Wind speed less than 10 mph.
- Zero cloudiness.


In [11]:
# Set the criteria for the ideal vacation weather condition

best_conditions_df = pd.DataFrame(weather_df, columns = ["City", 
                                                         "Country", 
                                                         "Latitude", 
                                                         "Longitude", 
                                                         "Max Temperature", 
                                                         "Wind Speed", 
                                                         "Cloudiness"])

good_max_temp = (best_conditions_df["Max Temperature"] <= 80) & (best_conditions_df["Max Temperature"] > 70)
good_wind_spd = best_conditions_df["Wind Speed"] < 10
good_cloudiness = best_conditions_df["Cloudiness"] == 0

hotel_df = best_conditions_df[good_max_temp & good_wind_spd & good_cloudiness]


hotel_df["Hotel Name"] = " "


## Hotel Map
Using Google Places API to find the first hotel for each city located within 5000 meters of your coordinates.  
- Store into variable named hotel_df.
- Add a "Hotel Name" column to the DataFrame.
- Set parameters to search for hotels with 5000 meters.
- Hit the Google Places API for each city's coordinates.
- Store the first Hotel result into the DataFrame.
 

In [12]:
# Loop through hotel_df with api calls
for index, row in hotel_df.iterrows():
    try:
        
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

        params = {
        "keyword": "hotel",
        "radius": 5000,
        "key": gkey,
        }


        lat = row["Latitude"]
        lng = row["Longitude"]

        params["location"] = f"{lat}, {lng}"

        hotel_data = requests.get(base_url, params=params).json()
        
        hotel_df.loc[index, "Hotel Name"] = hotel_data["results"][0]["name"]
        
    except IndexError:
        
        hotel_df.loc[index, "Hotel Name"] = "NaN"
      
hotel_df

,City,Country,Latitude,Longitude,Max Temperature,Wind Speed,Cloudiness,Hotel Name
1,Dubti,ET,11.74,41.09,76.21,6.51,0,NaN
46,Airai,TL,-8.93,125.41,71.02,1.25,0,NaN
49,Juan de Ayolas,PY,-27.40,-56.90,76.98,6.02,0,Playa Municipal De Ayolas
84,Salalah,OM,17.02,54.09,77.00,2.24,0,HYATT SALALAH HOTEL
111,Bogale,MM,16.28,95.40,73.60,7.49,0,IDEA Guest House
144,Mattru,SL,7.62,-11.83,74.64,2.06,0,NaN
148,Araouane,ML,18.90,-3.53,77.97,6.26,0,NaN
213,Flinders,AU,-34.58,150.86,72.00,3.00,0,Lakeview Hotel Motel
256,Yulara,AU,-25.24,130.99,78.80,8.05,0,Ayers Rock Resort
305,Faanui,PF,-16.48,-151.75,79.90,9.89,0,InterContinental Bora Bora Resort Thalasso Spa


In [21]:
# Drop the ones with NaN in Hotel Name column
hotel_df = hotel_df[hotel_df["Hotel Name"] != "NaN"]
hotel_df

,City,Country,Latitude,Longitude,Max Temperature,Wind Speed,Cloudiness,Hotel Name
49,Juan de Ayolas,PY,-27.40,-56.90,76.98,6.02,0,Playa Municipal De Ayolas
84,Salalah,OM,17.02,54.09,77.00,2.24,0,HYATT SALALAH HOTEL
111,Bogale,MM,16.28,95.40,73.60,7.49,0,IDEA Guest House
213,Flinders,AU,-34.58,150.86,72.00,3.00,0,Lakeview Hotel Motel
256,Yulara,AU,-25.24,130.99,78.80,8.05,0,Ayers Rock Resort
305,Faanui,PF,-16.48,-151.75,79.90,9.89,0,InterContinental Bora Bora Resort Thalasso Spa
335,Murud,IN,18.33,72.97,77.20,4.68,0,Murud Marina Hotel
466,Quelimane,MZ,-17.88,36.89,73.40,2.24,0,Hotel Elite
498,Ferkessédougou,CI,9.60,-5.20,74.62,4.41,0,Hotel Poufiret
551,Sur,OM,22.57,59.53,73.27,6.80,0,Sur Plaza Hotel


- Plot the hotels on top of the humidity heatmap with each pin containing the Hotel Name, City, and Country.

In [22]:
# Grabbing from stater_code
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""


# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [23]:

# Add marker layer ontop of heat map
# Display figure
markers = gmaps.marker_layer(locations, info_box_content= hotel_info)
fig.add_layer(markers)
fig



Figure(layout=FigureLayout(height='420px'))